In [3]:
! pip install onnx==1.11.0
! pip install onnxruntime==1.11.1
! pip install --user -e /workspace/qonnx

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///workspace/qonnx
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Attempting uninstall: qonnx
    Found existing installation: qonnx 0.0.post1.dev159+g627b4f6
    Uninstalling qonnx-0.0.post1.dev159+g627b4f6:
      Successfully uninstalled qonnx-0.0.post1.dev159+g627b4f6
  Running setup.py develop for qonnx


In [4]:
from qonnx.transformation.general import (
    ApplyConfig,
    GiveReadableTensorNames,
    GiveUniqueNodeNames,
    RemoveStaticGraphInputs,
    RemoveUnusedTensors,
)
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.fold_constants import FoldConstants

model = ModelWrapper("tinyyolo-20210831.onnx")
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())

In [5]:
import numpy as np
from qonnx.core.modelwrapper import ModelWrapper
from onnx import helper, TensorProto

def update_node(model, op_type):
    op_nodes = []
    for n in model.graph.node:
        if n.op_type == op_type:
            op_node = helper.make_node(
                op_type,
                inputs = n.input,
                outputs = n.output,
                name = n.name
            )
            if len(n.attribute) > 0:
                for na in n.attribute:
                    op_node.attribute.append(na)
#             model.graph.node.remove(n)
#             model.graph.node.append(new_bn_node)
            op_nodes.append(op_node)
    return op_nodes

def get_op_types(model):
    optype = []
    for n in m1.graph.node:
        optype.append(n.op_type)
    ops = list(np.unique(np.array(optype)))
    return ops

In [6]:
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.batchnorm_to_affine import BatchNormToAffine

model = ModelWrapper('tinyyolo-20210831.onnx')
gbl_in = model.graph.input[0]
gbl_out = model.graph.output[0]
ops = get_op_types(model)
# model.graph
nodes = []
for op in ops:
    for i in update_node(model,op):
        nodes.append(i) 
        
graph = helper.make_graph(
    nodes = nodes,
    inputs = model.graph.input,
    outputs = model.graph.output,
    initializer=model.graph.initializer,
    name = "new_nodes"
)
model = ModelWrapper(helper.make_model(graph))
# model = model.transform(InferShapes())
# model.model.opset_import[0].version = 9
# model.model.opset_import
model.save("updated_nodes.onnx")

model = model.transform(BatchNormToAffine())
# # model.graph.input

NameError: name 'm1' is not defined

In [14]:
model = ModelWrapper('tinyyolo-20210831_updated.onnx')
model = model.transform(InferShapes())
bn = update_node(model,"BatchNormalization")
for bn in bn:
    inp_shape = model.get_tensor_shape(bn.input[0])
    inp = helper.make_tensor_value_info(bn.input[0],TensorProto.FLOAT ,shape = inp_shape)
    opt = helper.make_tensor_value_info(bn.output[0],TensorProto.FLOAT ,shape = inp_shape)
    vi = []
    for i in bn.input[1:]:
        vi.append(model.get_tensor_valueinfo(i))
    graph = helper.make_graph(
            nodes = [bn],
            inputs = [inp],
            outputs = [opt],
            name = "BN_graph",    
            value_info= vi
    )

    bn_model = ModelWrapper(helper.make_model(graph))
    for i in bn.input[1:]:
        bn_model.set_initializer(i,model.get_initializer(i))

#     bn_model.model.ir_version = 6
#     bn_model.model.opset_import[0].version = 9
#     bn_model.model.opset_import
#     bn_model.save("sli_model.onnx")
    print(bn.name)
    bn_model.transform(BatchNormToAffine())


RuntimeError: Input 1 is out of bounds.